## Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

1. Start by creating a new Notebook for this assignment.
1. Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe like the one shown below:

In [1]:
import requests
import csv
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np


In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, header = 0)[0]

df.head()

Filter out 'Not assigned' values:

In [8]:
filtered_df = df[df['Borough'] != 'Not assigned']

filtered_df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


Group similar postal codes:

In [11]:
df_grouped = filtered_df.groupby(['Postal Code', 'Borough'], as_index = False).agg(lambda x: ", ".join(x))
df_grouped['Neighborhood'] = df_grouped['Neighbourhood'].str.replace('/', ',')

df_grouped.head()

,Postal Code,Borough,Neighbourhood,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge","Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn,Woburn
4,M1H,Scarborough,Cedarbrae,Cedarbrae
